In [ ]:
# !pip install flash-attn --no-build-isolation

In [1]:
import json
import outlines
import torch
from typing import List, Optional
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from datasets import load_dataset,Dataset
from rich import print
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Dict,Tuple,Any
import os
import tqdm
# os.environ["TRANSFORMERS_VERBOSITY"] = "info"
load_dotenv()

True

In [ ]:
MODEL_NAME = os.getenv("MODEL_NAME")
MODEL_PATH = os.getenv("MODEL_PATH")
DATASET_NAME = os.getenv("DATASET_NAME")
SEED = int(os.getenv("SEED"))
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 
GOOGLE_MODEL_ID = os.getenv("GOOGLE_MODEL_ID") 
GOOGLE_TEMPERATURE = float(os.getenv("GOOGLE_TEMPERATURE") )
GOOGLE_MAX_TOKENS = int(os.getenv("GOOGLE_MAX_TOKENS"))
EVAL_SET_SIZE = int(os.getenv("EVAL_SET_SIZE", 10))
MERGED_MODEL_OUTPUT_DIR = os.getenv("MERGED_MODEL_OUTPUT_DIR")
MODEL_FINETUNED_REPO_ID = os.getenv("MODEL_FINETUNED_REPO_ID")


In [3]:
import os
print(f"Current working directory: {os.getcwd()}")
absolute_model_path = os.path.abspath(MODEL_PATH)
print(f"Absolute model path: {absolute_model_path}")


Current working directory: /workspaces/Fine_Tuning_LLM/Code/eval-LLM

Absolute model path: /workspaces/Fine_Tuning_LLM/Models

In [4]:
print(f"Loading dataset: {DATASET_NAME}")
print(f"Using seed: {SEED}")

Loading dataset: Vishva007/RBI-Circular-QA-Dataset

Using seed: 42

In [5]:
try:
    dataset = load_dataset(DATASET_NAME,split="eval")
    print("Dataset loaded successfully!")
    print(dataset) # Print the dataset structure to see available splits
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure the dataset name is correct and you have an active internet connection.")


Dataset loaded successfully!

Dataset({
    features: ['document', 'filename', 'model_name', 'regulation_area', 'applicable_to', 'issued_on', 'key_topics',
'chunks_text', 'is_table', 'question', 'answer', 'evaluation_criteria', 'category', 'estimated_difficulty', 
'rephrased_question', 'rephrased_answer'],
    num_rows: 100
})

In [6]:
number_of_examples_for_eval = EVAL_SET_SIZE

# Get the total number of examples in full_dataset
num_examples = len(dataset)

# Calculate the start index for the last EVAL_SET_SIZE examples
start_index = num_examples - number_of_examples_for_eval

# Create a list of indices for the last 1EVAL_SET_SIZE0 examples
indices_to_select = list(range(start_index, num_examples))

# Use the .select() method to create a new Dataset object containing only those indices
eval_dataset = dataset.select(indices_to_select)


In [7]:
print("dataset type:", type(dataset))
print("eval_dataset type:", type(dataset))

dataset type: <class 'datasets.arrow_dataset.Dataset'>

eval_dataset type: <class 'datasets.arrow_dataset.Dataset'>

In [8]:
class ResponseFormat(BaseModel):
    answer : str
    
class EvaluationResult(BaseModel):
    score: int = Field(..., description="Score 1 if the answer fully satisfies ALL the specified criteria, 0 otherwise.", ge=0, le=1)


In [9]:
if torch.cuda.is_available():
    print("GPU is Used!")
else:
    print("CPU is Used!")

GPU is Used!

In [ ]:
model = outlines.models.transformers(MERGED_MODEL_OUTPUT_DIR,
                                     device="cuda",
                                     model_kwargs={"temperature":0.1,
                                                    "top_k":50,
                                                    "top_p":0.95,
                                                    "cache_dir":MERGED_MODEL_OUTPUT_DIR},)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

In [11]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
        model=GOOGLE_MODEL_ID, 
        temperature=GOOGLE_TEMPERATURE,
        max_tokens=GOOGLE_MAX_TOKENS,
        timeout=None,
        api_key=GOOGLE_API_KEY,
    )

In [5]:
def generate_answers_with_outlines(
    model,
    eval_dataset: Dataset,
    batch_size: int = 5,
    device: str = "cuda"
) -> List[Dict[str, str]]:
    """
    Generates answers for a dataset using an Outlines LLM.

    Args:
        model: The loaded HuggingFace model for Outlines.
        eval_dataset (datasets.Dataset): The dataset containing 'question' fields.
        batch_size (int): The number of samples to process in each batch.
        device (str): The device to run the model on (e.g., "cuda" or "cpu").

    Returns:
        List[Dict[str, str]]: A list of dictionaries, each containing the original question
                              and the 'generated_answer' from Outlines.
    """

    schema_as_str = json.dumps(ResponseFormat.model_json_schema())
    generator = outlines.generate.json(model, schema_as_str)

    generated_data = []

    print(f"Generating answers in batches of {batch_size}...")
    for i in range(0, len(eval_dataset), batch_size):
        batch_slice = eval_dataset.select(list(range(i, min(i + batch_size, len(eval_dataset)))))
        
        inputs = [item['question'] for item in batch_slice.to_list()]

        try:
            # Generate answers using outlines LLM
            outlines_outputs = generator(inputs)
            print(f"Generated {len(outlines_outputs)} answers for batch starting at index {i}.")
            print(outlines_outputs)
            
            # Combine original question with generated answer
            for original_item, outlines_output in zip(batch_slice.to_list(), outlines_outputs):
                generated_data.append({
                    "question": original_item['question'],
                    "generated_answer": outlines_output['answer'] 
                })
        except Exception as e:
            print(f"Error during Outlines generation for batch starting at index {i}: {e}")
            for original_item in batch_slice.to_list():
                 generated_data.append({
                    "question": original_item['question'],
                    "generated_answer": "\\ (Generation Failed)"
                })

    print("Outlines answer generation complete.")
    return generated_data

In [6]:
generated_answers = generate_answers_with_outlines(
    model,
    eval_dataset,
    batch_size=5,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

NameError: name 'eval_dataset' is not defined

In [18]:
print(generated_answers)

[
    {
        'question': 'What is the general role of the Banking Regulation Act of 1949 in the context of banking 
companies in India?',
        'generated_answer': 'Seven pillars of banking regulation in India: Advocacy, Coordination, Regulation, 
Supervision, Market, Enforcement and Aid. '
    },
    {
        'question': 'How should Nodal/Focal Point branches report April 2023 transactions, and how should they 
differentiate them from March Residual Transactions in their reporting?',
        'generated_answer': '}The company should create a separate folder in their Report tab for each applicable 
Nodal/Focal Point branch showing the report date for all transactions and specified date ranges of any 
developments.'
    },
    {
        'question': "What is the procedure for regulated entities to handle requests for delisting from the 
Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, and what alternative options are available for 
individuals or entities seeking removal from the list?",
        'generated_answer': 'Regulated entities handle requests for delisting through regular procedures in line 
with the United Nations Security Council, as well as access to alternative options through grassroots activists and
legal associations.'
    },
    {
        'question': 'How should Credit Information Companies provide individuals with access to their free annual 
credit report?',
        'generated_answer': "\\'Certain companies will keep your reports public and allow you to view them on the 
Federal Trade Commission website, while others will require a subscription or payment for access."
    },
    {
        'question': 'How can card issuing banks enable Card-on-File (CoFT) token generation, and what are the key 
requirements for customer consent and token availability?',
        'generated_answer': '\\'
    },
    {
        'question': 'What are the compliance requirements for Regulated Entities concerning the Unlawful Activities
(Prevention) Act (UAPA) Order?',
        'generated_answer': "Sreferences To Regulated Entities For Compliance With The Prevention Act, Order, 
Pursuant To Their Eligibilities As Personnel, Management, Agents, Or `Contractual Relationships' Under That Act."
    },
    {
        'question': 'Which bank is responsible for the lead bank responsibility in the Hojai district?',
        'generated_answer': 'Bank of Guyananehmen'
    },
    {
        'question': 'What steps should banks take to simplify the activation of inoperative or frozen accounts?',
        'generated_answer': "The full hands-on process would involve bank personnel to contact the account holder 
and then to follow up with specific steps including dialing the financial institution's phone number and of course 
dialing for the financial institution's customer support number, which then prompts the holder to click and enter 
specific information such as magnetic stripe card or account number, debit/credit card CVV number, store location, 
bank location, etc., and then complete the activation process once dialing in for the appropriate phone number, and
so forth. Helping bank professionals to review and advise on how to have users fix problems with 
inoperable/closed/overpaid/inactive/incompatible accounts, as well as banking issues of course, would be of value 
as well,"
    },
    {
        'question': "What is the purpose of the Reserve Bank of India's Master Direction on Financial Statements - 
Presentation and Disclosures?",
        'generated_answer': "The purpose of the Reserve Bank of India's Master Direction on Financial Statements - 
Presentation and Disclosures (MDFS) is to provide guidance and suggestions to regulators, advisory institutions, 
and the general public regarding the preparation of financial statements and disclosure materials."
    },
    {
        'question': 'What is the Reserve Bank of India\'s advice to participants in the Government Securities 
market regarding the use of the "Price / Yield rang

In [ ]:
def evaluate_answers_with_gemini(
    eval_dataset: Dataset,
    generated_answers_data: List[Dict[str, str]],
    llm: ChatGoogleGenerativeAI,
) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
    """
    Evaluates generated answers using a Gemini LLM, returning only the score in the structured output.

    Args:
        eval_dataset (datasets.Dataset): The original dataset containing 'question',
                                          'answer' (ground truth), and 'evaluation_criteria' fields.
        generated_answers_data (List[Dict[str, str]]): A list of dictionaries, where each
                                                       dictionary contains the original question
                                                       and the 'generated_answer'.
        llm (ChatGoogleGenerativeAI): The LangChain Gemini LLM instance.

    Returns:
        tuple: A tuple containing:
               - list: A list of dictionaries, where each dictionary contains the original question,
                       ground truth answer, generated answer, and evaluation score.
               - dict: A summary dictionary of the evaluation.
    """

    SYSTEM_MESSAGE = """
    You are an expert evaluator tasked with determining if an answer satisfies specified evaluation criteria.

    You will receive:
    1. A question
    2. The evaluation criteria
    3. The model's answer to evaluate

    Provide your score based on the criteria:
    - Score 1 if the answer fully satisfies ALL the specified criteria
    - Score 0 if it fails to meet ANY of the criteria

    Format your response as a JSON object with a single key 'score' containing just the number 1 or 0.
    For example: {{"score": 1}}
    """.strip()

    PROMPT_TEMPLATE = """
    Question: {question}
    Evaluation Criteria: {evaluation_criteria}
    Model Answer: {model_answer}

    Provide the evaluation score in the specified JSON format.
    """.strip()

    gemini_eval_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_MESSAGE),
            ("human", PROMPT_TEMPLATE),
        ]
    )

    gemini_eval_chain = gemini_eval_prompt | llm.with_structured_output(EvaluationResult)

    results = []
    total_evaluated = 0
    total_score = 0
    failed_evaluations_due_to_error = 0

    print(f"Evaluating answers with Gemini...")
    combined_data = list(zip(eval_dataset.to_list(), generated_answers_data))

    for original_item, generated_item in combined_data:
        total_evaluated += 1

        question = original_item['question']
        ground_truth_answer = original_item['answer']
        evaluation_criteria = original_item['evaluation_criteria']
        generated_answer = generated_item['generated_answer']

        gemini_input = {
            "question": question,
            "evaluation_criteria": evaluation_criteria,
            "model_answer": generated_answer,
        }

        score_val = 0

        try:
            structured_result = gemini_eval_chain.invoke(gemini_input)
            score_val = structured_result.score
            total_score += score_val

        except Exception as e:
            print(f"Error during Gemini evaluation for question: '{question}'. Error: {e}")
            score_val = 0
            failed_evaluations_due_to_error += 1

        results.append({
            "question": question,
            "ground_truth_answer": ground_truth_answer,
            "generated_answer": generated_answer,
            "evaluation_score": score_val,
        })

    # Calculate summary statistics
    successful_eval_count = total_evaluated - failed_evaluations_due_to_error
    percentage_passed_criteria = (total_score / successful_eval_count) * 100 if successful_eval_count > 0 else 0

    summary = {
        "total_evaluations_attempted": total_evaluated,
        "successful_evaluations": successful_eval_count,
        "failed_evaluations_due_to_error": failed_evaluations_due_to_error,
        "total_passed_criteria": total_score,
        "percentage_passed_criteria": percentage_passed_criteria,
        "overall_evaluation_summary": (
            f"Out of {total_evaluated} attempted evaluations, "
            f"{successful_eval_count} were successfully processed by Gemini. "
            f"{failed_evaluations_due_to_error} evaluations encountered an error. "
            f"Among the successfully evaluated answers, {total_score} met all criteria, "
            f"resulting in a {percentage_passed_criteria:.2f}% pass rate."
        )
    }
    print("Gemini evaluation complete.")
    return results, summary


In [16]:
evaluation_results, evaluation_summary = evaluate_answers_with_gemini(
    eval_dataset=eval_dataset, 
    generated_answers_data=generated_answers, 
    llm=llm,
)

Evaluating answers with Gemini...

In [17]:
print(evaluation_results)

[
    {
        'question': 'What is the general role of the Banking Regulation Act of 1949 in the context of banking 
companies in India?',
        'ground_truth_answer': 'The Banking Regulation Act of 1949 governs the functioning of banking companies in 
India, setting standards for their operations, management, and financial stability.',
        'generated_answer': 'Seven pillars of banking regulation in India: Advocacy, Coordination, Regulation, 
Supervision, Market, Enforcement and Aid. ',
        'evaluation_score': 0
    },
    {
        'question': 'How should Nodal/Focal Point branches report April 2023 transactions, and how should they 
differentiate them from March Residual Transactions in their reporting?',
        'ground_truth_answer': 'Nodal/Focal Point branches must submit a monthly statement for April transactions 
to Zonal Accounts Officers/Pay and Accounts Officers in the usual format. To differentiate between April 
transactions (financial year 2023-24) and March Residual Transactions, the statement pertaining to March Residual 
Transactions should be clearly marked as "March Residual Account".',
        'generated_answer': '}The company should create a separate folder in their Report tab for each applicable 
Nodal/Focal Point branch showing the report date for all transactions and specified date ranges of any 
developments.',
        'evaluation_score': 0
    },
    {
        'question': "What is the procedure for regulated entities to handle requests for delisting from the 
Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List, and what alternative options are available for 
individuals or entities seeking removal from the list?",
        'ground_truth_answer': "Regulated entities should forward any delisting requests they receive to the Joint 
Secretary (CTCR) at the Ministry of Home Affairs for consideration. Individuals or entities seeking removal from 
the Security Council's ISIL (Da'esh) and Al-Qaida Sanctions List can submit their request to an independent 
Ombudsperson appointed by the United Nations Secretary-General.",
        'generated_answer': 'Regulated entities handle requests for delisting through regular procedures in line 
with the United Nations Security Council, as well as access to alternative options through grassroots activists and
legal associations.',
        'evaluation_score': 0
    },
    {
        'question': 'How should Credit Information Companies provide individuals with access to their free annual 
credit report?',
        'ground_truth_answer': "Credit Information Companies must display a prominent link on their website's 
homepage, providing individuals easy access to their Free Full Credit Report, including their credit score, once 
per year between January and December.",
        'generated_answer': "\\'Certain companies will keep your reports public and allow you to view them on the 
Federal Trade Commission website, while others will require a subscription or payment for access.",
        'evaluation_score': 0
    },
    {
        'question': 'How can card issuing banks enable Card-on-File (CoFT) token generation, and what are the key 
requirements for customer consent and token availability?',
        'ground_truth_answer': "Card issuing banks can enable the generation of Card-on-File (CoFT) tokens through 
mobile and internet banking channels, provided explicit customer consent is obtained and validated using Additional
Factor Authentication (AFA). If a cardholder selects multiple merchants for tokenization, the AFA validation can be
combined for all of them. The generated tokens should then be available on the merchant's payment page within the 
cardholder's account.",
        'generated_answer': '\\',
        'evaluation_score': 0
    },
    {
        'question': 'What are the compliance requirements for Regulated Entities concerning the Unlawful Activities
(Prevention) Act (UAPA) Order?',
        'ground_truth_answer': 'Regulated Entities must adhere to th

In [19]:

print("\n--- Evaluation Summary ---")
summary_stats = evaluation_summary
print(f"Total Evaluations Attempted: {summary_stats['total_evaluations_attempted']}")
print(f"Evaluations Failed Due to Error: {summary_stats['failed_evaluations_due_to_error']}")
print(f"Total Answers Passed Criteria (Score 1): {summary_stats['total_passed_criteria']}")
print(f"Percentage of Successfully Evaluated Answers Passing Criteria: {summary_stats['percentage_passed_criteria']:.2f}%")
print(f"Overall Summary: {summary_stats['overall_evaluation_summary']}")

--- Evaluation Summary ---

Total Evaluations Attempted: 10

Evaluations Failed Due to Error: 0

Total Answers Passed Criteria (Score 1): 0

Percentage of Successfully Evaluated Answers Passing Criteria: 0.00%

Overall Summary: Out of 10 attempted evaluations, 10 were successfully processed by Gemini. 0 evaluations 
encountered an error. Among the successfully evaluated answers, 0 met all criteria, resulting in a 0.00% pass rate.